In [3]:
import random
import time
import spicy.stats as stats
# Obtener la hora actual en segundos desde el epoch
current_time = int(time.time())

In [4]:
import numpy as np
# Establece la semilla para la generación de números aleatorios en numpy usando la var
np.random.seed(current_time) # np.random.seed() inicializa el generador de números al
# Genera y muestra una muestra aleatoria de 10 elementos a partir del arreglo ["+", "-
# con probabilidades específicas para cada elemento
print(np.random.choice(["+", "-"], size=10, p=[0.5, 0.5]))
# np.random.choice() selecciona elementos aleatorios de la lista ["+", "-"]
# size=10 especifica que queremos una muestra de 10 elementos
# p=[0.5, 0.5] define la probabilidad de selección de cada elemento, en este caso, amb
# El resultado es un arreglo de 10 elementos, cada uno siendo "+" o "-" según las pro

['+' '-' '+' '+' '+' '-' '-' '+' '+' '+']


In [5]:
def runs_test(data):
 # Convertir los datos a una secuencia binaria para la prueba de runs
 median = np.median(data) # Calcula la mediana de los datos. La mediana se usa par
 binary_data = np.where(data > median, 1, 0) # Convierte los datos en una secuenci

 # Contar el número de "runs" (secuencias consecutivas de 0s o 1s)
 runs = np.count_nonzero(np.diff(binary_data) != 0) + 1
 # np.diff(binary_data) calcula la diferencia entre elementos consecutivos de binar
 # np.count_nonzero() cuenta cuántas diferencias son distintas de 0, es decir, cuán
 # +1 se agrega para incluir el primer run.
 # Número total de 0s y 1s
 n0 = np.sum(binary_data == 0) # Cuenta el número de 0s en binary_data
 n1 = np.sum(binary_data == 1) # Cuenta el número de 1s en binary_data

 # Calcular la estadística de la prueba y el valor p
 expected_runs = 1 + (2 * n0 * n1) / (n0 + n1)
 # La fórmula para el número esperado de runs en una secuencia binaria es 1 + (2 *
 variance_runs = (expected_runs - 1) * (expected_runs - 2) / (n0 + n1 - 1)
 # La varianza de los runs se calcula con la fórmula (expected_runs - 1) * (expecte
 z = (runs - expected_runs) / np.sqrt(variance_runs)
 # La estadística z se calcula como (runs - expected_runs) / sqrt(variance_runs), q
 p_value = 2 * (1 - stats.norm.cdf(np.abs(z)))
 # El valor p se calcula usando la distribución normal estándar. stats.norm.cdf() c
 # 2 * (1 - CDF) se usa para obtener el valor p bilateral, que indica la probabilid
 return runs, expected_runs, variance_runs, z, p_value
 # Devuelve el número observado de runs, el número esperado de runs, la varianza de
# Ejemplo de uso
data = np.random.random(100) # Genera un array de 100 datos aleatorios entre 0 y 1
runs, expected_runs, variance_runs, z, p_value = runs_test(data)
# Llama a la función runs_test con los datos generados y guarda los resultados en las
print(f'Número de runs: {runs}')
print(f'Número esperado de runs: {expected_runs}')
print(f'Varianza de runs: {variance_runs}')
print(f'Estadística z: {z}')
print(f'Valor p: {p_value}')
# Imprime el número de runs observado, el número esperado de runs, la varianza de los

Número de runs: 53
Número esperado de runs: 51.0
Varianza de runs: 24.747474747474747
Estadística z: 0.402035636556294
Valor p: 0.6876577997456863


In [7]:
import numpy as np
import scipy.stats as stats

def anderson_darling_uniform_test(data):
    """
    Realiza la prueba de bondad de ajuste de Anderson-Darling para verificar si los datos 
    siguen una distribución uniforme.

    Parameters:
    data (array-like): Muestra de datos a ser probada.

    Returns:
    A2 (float): Estadístico de la prueba de Anderson-Darling.
    p_value (float): Valor p estimado a partir de la simulación.
    """
    # Asegurarse de que los datos estén en el rango [0, 1]
    data = np.clip(data, 0, 1)

    # Ordenar los datos de menor a mayor
    sorted_data = np.sort(data)
    n = len(data)  # Número de datos en la muestra

    # Calcular el estadístico de Anderson-Darling para la distribución uniforme
    # Para la distribución uniforme, F(x) = x (CDF uniforme)
    cdf_values = sorted_data
    # Term1 es la suma de los logaritmos de los valores CDF y su complemento
    term1 = -np.sum((2 * np.arange(1, n + 1) - 1) * (np.log(cdf_values) + np.log(1 - cdf_values)))
    term2 = -n  # Término de ajuste

    # Calcula el estadístico A2
    A2 = (term1 / n) + term2

    # Aproximar el valor p usando simulaciones para crear una distribución de referencia
    num_simulations = 10000  # Número de simulaciones para aproximar la distribución de A2
    simulated_statistics = []

    for _ in range(num_simulations):
        # Generar datos simulados uniformemente distribuidos
        simulated_data = np.random.uniform(0, 1, size=n)
        simulated_sorted_data = np.sort(simulated_data)
        simulated_cdf_values = simulated_sorted_data
        
        # Calcular el estadístico de Anderson-Darling para los datos simulados
        simulated_term1 = -np.sum((2 * np.arange(1, n + 1) - 1) * (np.log(simulated_cdf_values) + np.log(1 - simulated_cdf_values)))
        simulated_term2 = -n
        simulated_A2 = (simulated_term1 / n) + simulated_term2
        
        simulated_statistics.append(simulated_A2)

    simulated_statistics = np.array(simulated_statistics)
    # Calcular el valor p como la proporción de estadísticos simulados que son mayores o iguales a A2
    p_value = np.mean(simulated_statistics >= A2)

    return A2, p_value


In [8]:
# Ejemplo de uso
np.random.seed(42) # Establece una semilla para reproducibilidad de los resultados
data = np.random.uniform(0, 1, size=100) # Generar una muestra de 100 números aleator
# Realizar la prueba de Anderson-Darling
A2, p_value = anderson_darling_uniform_test(data)
# Mostrar los resultados
print(f'Estadística de Anderson-Darling: {A2}')
print(f'Valor p: {p_value}')
# Evaluar el resultado basándose en un nivel de significancia del 5%
alpha = 0.05
if p_value < alpha:
 print('La prueba sugiere que los datos no siguen una distribución uniforme.')
else:
 print('La prueba sugiere que los datos siguen una distribución uniforme.')

Estadística de Anderson-Darling: 95.08737739498511
Valor p: 0.6503
La prueba sugiere que los datos siguen una distribución uniforme.


In [16]:
k=10
u=np.random.random()
x=int(k*u)+1
print(x)
print(u)

5
0.4050332388498735
